<a href="https://colab.research.google.com/github/simulate111/General/blob/main/Copenhagen_Denmark_NASAPower.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
#pip install meteostat

In [35]:
import requests
import pandas as pd
import time

# --- CONFIGURATION ---
# Copenhagen, Denmark
LAT = 55.6761
LON = 12.5683

# NASA POWER Date Format: YYYYMMDD
START_DATE = "20240101"
END_DATE = "20241231"

# API Endpoint for Hourly Point Data
base_url = "https://power.larc.nasa.gov/api/temporal/hourly/point"

# Parameters Definition:
# T2M                = Temperature at 2 Meters (C)
# WS10M              = Wind Speed at 10 Meters (m/s)
# ALLSKY_SFC_SW_DWN  = Global Horizontal Irradiance (GHI) - W/m^2
# ALLSKY_SFC_SW_DNI  = Direct Normal Irradiance (DNI) - W/m^2
# ALLSKY_SFC_SW_DIFF = Diffuse Horizontal Irradiance (DHI) - W/m^2
params = {
    "parameters": "T2M,WS10M,ALLSKY_SFC_SW_DWN,ALLSKY_SFC_SW_DNI,ALLSKY_SFC_SW_DIFF",
    "community": "RE", # Renewable Energy Community
    "longitude": LON,
    "latitude": LAT,
    "start": START_DATE,
    "end": END_DATE,
    "format": "JSON"
}

print(f"Fetching NASA POWER Data for Copenhagen ({START_DATE}-{END_DATE})...")
print("This may take 10-20 seconds as NASA processes the request.")

try:
    r = requests.get(base_url, params=params)

    if r.status_code == 200:
        data = r.json()

        # NASA returns a nested dictionary structure
        # We need to flatten it. The keys are YYYYMMDDHH
        properties = data['properties']['parameter']

        # Get the list of timestamps from one valid parameter (e.g., T2M)
        timestamps = sorted(properties['T2M'].keys())

        records = []
        for ts in timestamps:
            records.append({
                'Time_Str': ts,
                'Temperature_C': properties['T2M'].get(ts),
                'Wind_Speed_ms': properties['WS10M'].get(ts),
                'GHI_Wm2': properties['ALLSKY_SFC_SW_DWN'].get(ts),
                'DNI_Wm2': properties['ALLSKY_SFC_SW_DNI'].get(ts),
                'DHI_Wm2': properties['ALLSKY_SFC_SW_DIFF'].get(ts)
            })

        df = pd.DataFrame(records)

        # Convert Timestamp String (YYYYMMDDHH) to DateTime Object
        df['Time'] = pd.to_datetime(df['Time_Str'], format='%Y%m%d%H')

        # Formatting Columns
        df['Date'] = df['Time'].dt.strftime('%Y-%m-%d')
        df['Hour'] = df['Time'].dt.strftime('%H:%M')

        # Cleanup: NASA uses -999 for missing data
        df = df.replace(-999, pd.NA)

        # Reorder for final CSV
        cols = ['Date', 'Hour', 'Temperature_C', 'Wind_Speed_ms', 'GHI_Wm2', 'DNI_Wm2', 'DHI_Wm2']
        df = df[cols]

        print("-" * 30)
        print(df.head())
        print("-" * 30)
        print(f"Total Rows: {len(df)}")

        filename = "Copenhagen_NASA_POWER_2024.csv"
        df.to_csv(filename, index=False)
        print(f"Success! Data saved to {filename}")

    else:
        print(f"Error {r.status_code}: {r.text}")

except Exception as e:
    print(f"Connection failed: {e}")

Fetching NASA POWER Data for Copenhagen (20240101-20241231)...
This may take 10-20 seconds as NASA processes the request.
------------------------------
         Date   Hour  Temperature_C  Wind_Speed_ms  GHI_Wm2  DNI_Wm2  DHI_Wm2
0  2024-01-01  00:00           4.92           4.85      0.0      0.0      0.0
1  2024-01-01  01:00           4.83           4.08      0.0      0.0      0.0
2  2024-01-01  02:00           4.61           3.65      0.0      0.0      0.0
3  2024-01-01  03:00           4.28           3.67      0.0      0.0      0.0
4  2024-01-01  04:00           3.92           3.85      0.0      0.0      0.0
------------------------------
Total Rows: 8784
Success! Data saved to Copenhagen_NASA_POWER_2024.csv
